In [1]:
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import pandas as pd

tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [2]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
html_data = response.text

soup = BeautifulSoup(html_data, "html.parser")

tables = soup.find_all("table")

for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        revenue_table = table
        break

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in revenue_table.tbody.find_all("tr"):
    cols = row.find_all("td")
    if cols:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if revenue:  # 排除空值
            tesla_revenue = pd.concat(
                [tesla_revenue, pd.DataFrame([[date, revenue]], columns=["Date", "Revenue"])],
                ignore_index=True
            )

tesla_revenue.tail()

,Date,Revenue
57,2010-09-30,31
58,2010-06-30,28
59,2010-03-31,21
60,2009-09-30,46
61,2009-06-30,27


In [ ]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")

gme_data.reset_index(inplace=True)
gme_data.head()

In [ ]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
html_data = response.text

soup = BeautifulSoup(html_data, "html.parser")
tables = soup.find_all("table")

for table in tables:
    if "GameStop Quarterly Revenue" in table.text:
        revenue_table = table
        break

gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in revenue_table.tbody.find_all("tr"):
    cols = row.find_all("td")
    if cols:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if revenue:
            gme_revenue = pd.concat(
                [gme_revenue, pd.DataFrame([[date, revenue]], columns=["Date", "Revenue"])],
                ignore_index=True
            )

gme_revenue.tail()

In [ ]:
import matplotlib.pyplot as plt

def make_graph(stock_data, revenue_data, stock, revenue):
    fig, ax1 = plt.subplots(figsize=(14, 6))

    stock_data["Date"] = pd.to_datetime(stock_data["Date"]).dt.tz_localize(None)
    revenue_data["Date"] = pd.to_datetime(revenue_data["Date"])

    ax1.plot(stock_data['Date'], stock_data['Close'], label="Stock Price", color='blue')
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Stock Price", color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    ax2 = ax1.twinx()
    ax2.plot(revenue_data['Date'], revenue_data['Revenue'].astype(float), label="Revenue", color='green')
    ax2.set_ylabel("Revenue", color='green')
    ax2.tick_params(axis='y', labelcolor='green')

    plt.title(f"{stock} Stock Price and {revenue} Revenue Over Time")
    plt.show()

make_graph(tesla_data, tesla_revenue, 'Tesla', 'Tesla')

In [ ]:
make_graph(gme_data, gme_revenue, 'GameStop', 'GameStop')